In [1]:
import os
import numpy as np
import glob
import pandas as pd  
import random
import math as mp
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import time

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow.keras.backend as K
import toml

from tensorflow.keras.models import Sequential
from keras.models import model_from_yaml
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalMaxPooling1D

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from pyquaternion import Quaternion

from numpy import array
from numpy import hstack

from math import floor

from ipynb.fs.full.V06_Utils  import *

In [77]:
######################################################################
# Load Configuration
######################################################################

config_path = 'C:/Users/weikert1/Documents/Thesis/Project_V06_26072021/Code/V06_config.toml'
cfg = toml.load(config_path)


######################################################################
# Load Directories 
######################################################################

# Predicted Traces
Path_dataset01_test = cfg['Path_dataset01']
Path_dataset02_test = cfg['Path_dataset02']

# LSTM quat 
Path_results_dataset01_LSTM_quat = cfg['Path_results_dataset01'] + "LSTM_quat/"
Path_results_dataset02_LSTM_quat = cfg['Path_results_dataset02'] + "LSTM_quat/"

# LSTM euler
Path_results_dataset01_LSTM_euler = cfg['Path_results_dataset01'] + "LSTM_euler/"
Path_results_dataset02_LSTM_euler = cfg['Path_results_dataset02'] + "LSTM_euler/"

# GRU euler
Path_results_dataset01_GRU_euler = cfg['Path_results_dataset01'] + "GRU_euler/"
Path_results_dataset02_GRU_euler = cfg['Path_results_dataset02'] + "GRU_euler/"

# Baseline euler
Path_results_dataset01_Baseline_euler = cfg['Path_results_dataset01'] + "Baseline_euler/"
Path_results_dataset02_Baseline_euler = cfg['Path_results_dataset02'] + "Baseline_euler/"

# Baseline quat
Path_results_dataset01_Baseline_quat = cfg['Path_results_dataset01'] + "Baseline_quat/"
Path_results_dataset02_Baseline_quat = cfg['Path_results_dataset02'] + "Baseline_quat/"

# Tensorflow configuration
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  1


# Result Directories

In [78]:
############################
# LSTM quat
############################
model_ID = cfg['1_2_model_ID'] 
Path_results_dataset01_LSTM_quat_model_id_evaluation =  Path_results_dataset01_LSTM_quat + "/" + model_ID + "/Evaluation/" 
Path_results_dataset02_LSTM_quat_model_id_evaluation =  Path_results_dataset02_LSTM_quat + "/" + model_ID + "/Evaluation/" 

############################
# LSTM euler
############################
model_ID = cfg['2_1_model_ID'] 
Path_results_dataset01_LSTM_euler_model_id_evaluation =  Path_results_dataset01_LSTM_euler + "/" + model_ID + "/Evaluation/" 
Path_results_dataset02_LSTM_euler_model_id_evaluation =  Path_results_dataset02_LSTM_euler + "/" + model_ID + "/Evaluation/"

############################
# GRU euler
############################
model_ID = cfg['4_1_model_ID'] 
Path_results_dataset01_GRU_euler_model_id_evaluation =  Path_results_dataset01_GRU_euler + "/" + model_ID + "/Evaluation/" 
Path_results_dataset02_GRU_euler_model_id_evaluation =  Path_results_dataset02_GRU_euler + "/" + model_ID + "/Evaluation/" 

############################
# Baseline euler
############################

Path_results_dataset01_Baseline_euler_model_id_evaluation =  Path_results_dataset01_Baseline_euler + "/" + "/Evaluation/" 
Path_results_dataset02_Baseline_euler_model_id_evaluation =  Path_results_dataset02_Baseline_euler + "/" + "/Evaluation/" 

# Evaluation Euler

In [92]:
list_datasets = ['dataset_02']
list_models = ['GRU_euler', 'LSTM_euler', 'Baseline']

In [93]:
frames = []
def get_interim_results(path,dataset,model):
    for csv in get_csv_files(path, "*evaluation"):
        #print(csv)
        basename = os.path.splitext(os.path.basename(csv))[0]
        csv_df = pd.read_csv(csv, index_col=None, header=0)
        add_basename = [basename] 
        add_dataset = [dataset]
        add_model =[model]
        csv_df['basename'] = add_basename
        csv_df['dataset'] = add_dataset
        csv_df['model'] = add_model
        csv_df = csv_df[['dataset', 'model', 'basename', 'mae_ang_pitch', 'mae_ang_roll', 'mae_ang_yaw']]
        frames.append(csv_df)
    result = pd.concat(frames, axis=0, ignore_index=True)
    return result 

def get_results():
    for dataset in list_datasets:
        for model in list_models:
            if model == "GRU_euler":
                if dataset == "dataset_01":
                    path = Path_results_dataset01_GRU_euler_model_id_evaluation
                    result = get_interim_results(path, dataset, model)
                if dataset == "dataset_02":
                    path = Path_results_dataset02_GRU_euler_model_id_evaluation
                    result = get_interim_results(path, dataset, model)
            if model == "LSTM_euler":
                if dataset == "dataset_01":
                    path = Path_results_dataset01_LSTM_euler_model_id_evaluation
                    result = get_interim_results(path, dataset, model)
                if dataset == "dataset_02":
                    path = Path_results_dataset02_LSTM_euler_model_id_evaluation
                    result = get_interim_results(path, dataset, model)
            if model == "Baseline":
                if dataset == "dataset_01":
                    path = Path_results_dataset01_Baseline_euler_model_id_evaluation
                    result = get_interim_results(path, dataset, model)
                if dataset == "dataset_02":
                    path = Path_results_dataset02_Baseline_euler_model_id_evaluation
                    result = get_interim_results(path, dataset, model)
    return result


result = get_results()
result.to_excel(os.path.join(cfg['Path_summary_results'], 'summary_results_euler_dataset02.xlsx'), index=False)


# Evaluation Quaternion

In [91]:
result

,dataset,model,basename,mae_ang_pitch,mae_ang_roll,mae_ang_yaw
0,dataset_01,GRU_euler,1_test_result_euler_lstm_evaluation,3.539182,5.589566,3.857692
1,dataset_01,GRU_euler,2_test_result_euler_lstm_evaluation,3.904097,1.057998,3.168077
2,dataset_01,GRU_euler,3_test_result_euler_lstm_evaluation,5.051243,3.576532,4.322400
3,dataset_01,GRU_euler,5_test_result_euler_lstm_evaluation,3.599665,3.257677,2.876378
4,dataset_01,LSTM_euler,1_test_result_euler_lstm_evaluation,6.374745,13.913673,5.721469
5,dataset_01,LSTM_euler,2_test_result_euler_lstm_evaluation,7.786124,1.956451,4.645992
6,dataset_01,LSTM_euler,3_test_result_euler_lstm_evaluation,10.052531,8.295125,6.420311
7,dataset_01,LSTM_euler,5_test_result_euler_lstm_evaluation,6.130692,8.523539,4.453306
8,dataset_01,Baseline,1_test_result_rot_baseline_evaluation,1.547388,0.818611,2.635339
9,dataset_01,Baseline,2_test_result_rot_baseline_evaluation,1.698419,0.548735,2.245339
